## Adicionando o path para utilizar a bibliotecas necessárias

In [1]:
from sys import path
dir_path: str = %pwd
dir_path = dir_path.replace("/Simulations", "")

path.append(dir_path)

## Fazendo import das dependências

In [2]:
from BHA_functions import asyncSimulations_Linux
from BHA_functions import DataCollector, DataGroup
from BHA_functions import GraphicGenerator
from datetime import datetime

## Definindo as constantes das simulações

In [3]:
PATH: str = f'{dir_path}/Simulations_Data/default_simulations'
GRAPHICS_PATH: str = f"{dir_path}/Graphics/default_simulations/"
CORES: int = 12
RUNS_PER_POINT: int = 1_000

TARGET: tuple[bool, bool] = (True, False)
GRAPHIC_POINTS: int = 7

NETWORK_PROB: float = 0.8
TOPOLOGY: str = 'Grade'
INTENSITY_VARIATION: float = 0.1
BHA_NODES: tuple[int, int, int] = (1, 3, 5)

## Rodando a simulação sem Black Hole

In [ ]:
intensity: list = []
for c in range(1, GRAPHIC_POINTS+1):
    intensity.append(NETWORK_PROB - c*INTENSITY_VARIATION)

# Default params
simulations_params: dict = {
            'runs':RUNS_PER_POINT,
            'topology':TOPOLOGY,
            'number_nodes':20,
            'topology_args':(4, 3),
            'entanglements_replanished':10,
            'requests':100,
            'attempts_per_request':2,
            'network_prob':NETWORK_PROB,
            'num_black_holes':0,
            'black_hole_prob':intensity[0],
            'black_hole_target':TARGET
        }

# Runing no BHA simulation
default_simulation: DataCollector = asyncSimulations_Linux(cores=CORES, **simulations_params)
default_simulation.save(file_name=f'{PATH}/Default_Network/default_network')

## Realizando todas as simulações com Black Holes

In [ ]:
counter = 0
for target in TARGET:
    # Updating parameter
    simulations_params['black_hole_target'] = target

    for bha_nodes in BHA_NODES:
        # Updating the parameter
        simulations_params['num_black_holes'] = bha_nodes

        for i in range(0, GRAPHIC_POINTS):
            start: datetime = datetime.now()

            # Updating the parameter
            simulations_params['black_hole_prob'] = intensity[i]

            # Running simulation
            simulations_dc: DataCollector = asyncSimulations_Linux(cores=CORES, **simulations_params)

            # Saving CSV file
            simulations_dc.save(file_name=f'{PATH}/Target-{target}/{bha_nodes}nodes/point{i}')

            counter += 1

            # BenchMark
            print(f"As {simulations_params['runs']} simulações com target {target} e com {bha_nodes} atacantes finalizaram no tempo de: {datetime.now()-start}")
            print(f"Foram gerados {counter} pontos do total de 42")
            print('-='*50)

## Coletando os dados diretamente dos CSVs de cada simulação

In [6]:
# DataCollector of no BHA network
default_network: DataCollector = DataCollector()
default_network.get_DataFrame_csv(f'{PATH}/Default_Network/default_network.csv')

# DataGroup to storage all data
dataGroup_target_on: DataGroup = DataGroup()
dataGroup_target_off: DataGroup = DataGroup()

# Will collect and separete with target on and off
for target in TARGET:
    for indexgroup, bha_nodes in enumerate(BHA_NODES):
        for i in range(0, GRAPHIC_POINTS):
            temp_dc: DataCollector = DataCollector()
            temp_dc.get_DataFrame_csv(f'{PATH}/Target-{target}/{bha_nodes}nodes/point{i}.csv')

            if target:
                dataGroup_target_on.add_Data(value=temp_dc, indexgroup=indexgroup)
            else:
                dataGroup_target_off.add_Data(value=temp_dc, indexgroup=indexgroup)

## Criando uma função genérica para plotar os gráficos

In [7]:
def plotGraphic(
        dataGroup: DataGroup,
        default_diff: DataCollector, 
        intensity_variation: float,
        bha_nodes: tuple,
        y_column_name: str, 
        title: str, 
        x_label: str, 
        y_label: str,
        pdf_file: str,
        grid: bool) -> None:
    """
    Will show the simulations graph

    Args:
        dataGroup (required): DataGroup with all DataCollectors
        default_diff (required): Pass the network without black holes to compare, if None don't affect the result
        intensity_variation (required): Variation of intensity per point
        bha_nodes (required): Tuple with number of nodes
        y_column_name (required): Name of column data
        title (required): Title of graph
        x_label (required): X axis label
        y_label (required): Y axis label
        pdf_file (optional): Name of pdf file, if don't want save as pdf, pdf_file = ""
        grid (optional): Add a grid on the plot if is True
    """
    
    graphicGen: GraphicGenerator = GraphicGenerator()

    colors: tuple[str, str, str] = ("red", "darkorange", "gold")

    for pos, datacollectors in enumerate(dataGroup):
        if bha_nodes[pos] == 1:
            node_str: str = "nó"
        else:
            node_str: str = "nós"

        graphicGen.add_on_plot(
            plot_label=f'{bha_nodes[pos]} {node_str}',
            color=colors[pos],
            x_column=(0.1, intensity_variation), 
            y_column_name=y_column_name, 
            y_standard_deviation=False,
            dc=datacollectors,
            default_diff=default_diff)

    graphicGen.show_plot(
        title=title,
        x_label=x_label, 
        y_label=y_label,
        pdf_file=pdf_file,
        grid=grid)

## Gerando o gráfico da Taxa de Sucesso sem Target

In [ ]:
plotGraphic(
    dataGroup=dataGroup_target_off,
    default_diff=default_network,
    intensity_variation=INTENSITY_VARIATION,
    bha_nodes=BHA_NODES,
    y_column_name="Success Tax",
    title="",
    x_label='Intensidade',
    y_label='Diferença na Taxa de Sucesso',
    pdf_file=f"{GRAPHICS_PATH}Taxa_Sucesso_sem_Target.pdf",
    grid=True
    )

## Gerando o gráfico da Taxa de Sucesso com Target

In [ ]:
plotGraphic(
    dataGroup=dataGroup_target_on,
    default_diff=default_network,
    intensity_variation=INTENSITY_VARIATION,
    bha_nodes=BHA_NODES,
    y_column_name="Success Tax",
    title="",
    x_label='Intensidade',
    y_label='Diferença na Taxa de Sucesso',
    pdf_file=f"{GRAPHICS_PATH}Taxa_Sucesso_com_Target.pdf",
    grid=True
    )

## Gerando o Gráfico da Taxa de Eprs Usados sem Target

In [ ]:
plotGraphic(
    dataGroup=dataGroup_target_off,
    default_diff=default_network,
    intensity_variation=INTENSITY_VARIATION,
    bha_nodes=BHA_NODES,
    y_column_name="Used Eprs",
    title="",
    x_label='Intensidade',
    y_label='Diferença na Taxa de Eprs Usados',
    pdf_file=f"{GRAPHICS_PATH}Taxa_Eprs_Usados_sem_Target.pdf",
    grid=True
)

## Gerando o Gráfico da Taxa de Eprs Usados com Target

In [ ]:
plotGraphic(
    dataGroup=dataGroup_target_on,
    default_diff=default_network,
    intensity_variation=INTENSITY_VARIATION,
    bha_nodes=BHA_NODES,
    y_column_name="Used Eprs",
    title="",
    x_label='Intensidade',
    y_label='Diferença na Taxa de Eprs Usados',
    pdf_file=f"{GRAPHICS_PATH}Taxa_Eprs_Usados_com_Target.pdf",
    grid=True
)

## Gerando o Gráfico da Fidelidade Média das Rotas sem Target

In [ ]:
plotGraphic(
    dataGroup=dataGroup_target_off,
    default_diff=default_network,
    intensity_variation=INTENSITY_VARIATION,
    bha_nodes=BHA_NODES,
    y_column_name="Avg Fidelity Route",
    title="",
    x_label='Intensidade',
    y_label='Diferença na Fidelidade Média da Rota',
    pdf_file=f"{GRAPHICS_PATH}Fidelidade_Média_Rotas_sem_Target.pdf",
    grid=True
)

## Gerando o Gráfico da Fidelidade Média das Rotas com Target

In [ ]:
plotGraphic(
    dataGroup=dataGroup_target_on,
    default_diff=default_network,
    intensity_variation=INTENSITY_VARIATION,
    bha_nodes=BHA_NODES,
    y_column_name="Avg Fidelity Route",
    title="",
    x_label='Intensidade',
    y_label='Diferença na Fidelidade Média da Rota',
    pdf_file=f"{GRAPHICS_PATH}Fidelidade_Média_Rotas_com_Target.pdf",
    grid=True
)